Definición poset

In [3]:
def generar_poset(ELEMENTOS,CARD_CONJ):
    covers=dict()
    familia=[tuple(sorted(conj)) for conj in Subsets(ELEMENTOS, CARD_CONJ)]
    for conj in familia:
        cov=[]
        for i in range(CARD_CONJ):
            agregar=tuple([conj[j] if i!= j else conj[j]+1 for j in range(CARD_CONJ)])
            if agregar in familia:
                cov.append(agregar)
        covers[conj]=cov
    return Poset(covers)

Encontrar ideales y tuplas minimales

In [4]:
def encontrar_min(GENERADORES,P):
    ideal_gen = P.order_ideal(GENERADORES)
    gen_ideal_cont=[]
    for i in range(len(GENERADORES[0])):
        e= max(t[i] for t in GENERADORES)
        gen_ideal_cont.append(e)
    ideal_cont = P.order_ideal([tuple(gen_ideal_cont)])
    elem_min=[]
    for conj in ideal_cont:
        if not(conj in ideal_gen):
            minimal =True
            for e in P.lower_covers(conj):
                if not(e in ideal_gen):
                    minimal=False
            if minimal:
                elem_min.append(conj)
    return elem_min

Generar cono en el que elementos minimales son mayores que los generadores

In [5]:
def generar_cono(ELEMENTOS,GENERADORES, elem_min):   
    desigualdades=[]
    for conj_max in GENERADORES:
        for conj_min in elem_min:
            pos=[1 if i in conj_min else 0 for i in range(ELEMENTOS+1)]
            neg=[-1 if i in conj_max else 0 for i in range(ELEMENTOS+1)]
            res=tuple([pos[i]+neg[i]for i in range(ELEMENTOS+1)])
            desigualdades.append(res)
    for i in range(1,ELEMENTOS):
        desigualdades.append(tuple([-1 if i==j else 1 if i+1==j else 0 for j in range(ELEMENTOS+1)]))
    return Polyhedron(ieqs=desigualdades)

Función que recibe poset y cantidad de generadores y devuelve anticadenas del largo correspondiente

In [6]:
def generadores_ideales(P,CANT_GEN):
    gens_pos=[]
    for conj in Subsets(P.list(), CANT_GEN):
        if P.is_antichain_of_poset(conj):
            gens_pos.append(conj)
    return gens_pos

Función que recibe un poset y devuelve una lista de las tuplas de grado maximal

In [23]:
from operator import countOf
from collections import defaultdict
from copy import deepcopy

def menor_que(menor, mayor):
    #return True si menor es menor que mayor, sino, devuelve False
    if menor==mayor:
        return False
    for i in range(len(menor)):
        if sum(menor[:i])>sum(mayor[:i]):
            return False
    return True

def calcular_grado(P,generadores, ELEMENTOS):
    ideal = P.order_ideal(generadores)
    lista_elementos = [e for conj in ideal for e in conj]
    grado = tuple([countOf(lista_elementos,i+1) for i in range(ELEMENTOS)])
    return len(ideal), grado

def degree_max(P,ELEMENTOS):
    dict_grados = defaultdict(set)
    anticadenas = P.antichains()
    for anticad in anticadenas:
#         print(anticad)
        cant_elementos, grado = calcular_grado(P,anticad,ELEMENTOS)
        dict_grados[cant_elementos].add(grado)
    dict_degree_max = deepcopy(dict_grados)
#     print(f'dict 1 listo con {len(dict_grados.keys())} llaves')
    for i in dict_grados.keys():
#         print(i)
        for sec_grado in dict_grados[i]:
            no_eliminado=True
            for otra_sec in dict_grados[i]:
                if no_eliminado and menor_que(sec_grado,otra_sec):
#                     if sec_grado in [(23,19,16,16,12,10,10,7,4),(22,18,18,15,15,12,11,6,3),(23,20,18,18,15,12,11,7,5),(23,16,16,12,12,12,7,7,3),
#                                     (23,18,18,15,15,12,12,7,3),(24,19,16,15,11,11,8,8,5),(24,20,18,16,13,11,11,8,5),(24,19,16,16,12,12,8,8,5),
#                                     (24,20,18,18,15,12,12,8,5)]:
#                         print(sec_grado,otra_sec)
                    dict_degree_max[i].remove(sec_grado)
                    no_eliminado=False
    print(dict_grados==dict_degree_max)
    return dict_degree_max
    
P=generar_poset(9,3)
dict_grados=degree_max(P,9)

dict 1 listo con 85 llaves
False


Función recibe cantidad de elementos, cardinalidad de conjuntos y conatidad de generadores. El output son los generadores que no son M-threshold en una lista de tuplas (generadores, # de repeticiones)

In [24]:
import random
from operator import countOf

def experimentos(ELEMENTOS,CARD_CONJ, CANT_GEN):
    datos=[['Generadores', 'dimensión cono', 'Cantidad de igualdades en rayos']]
    P=generar_poset(ELEMENTOS,CARD_CONJ)
    gens_pos = generadores_ideales(P,CANT_GEN)
    gens_interesantes=[]
    for gen in gens_pos:
        elem_min = encontrar_min(gen,P)
        cono = generar_cono(ELEMENTOS, gen, elem_min)
        
        #COMENTAR DESDE AQUÍ PARA OBTENER TABLA CON TODOS LOS DATOS        
        elemento_cono=sum([random.randint(1,100)*vector(rayo) for rayo in cono.rays()])
        prod_pto_gen = [sum([elemento_cono[i-1] for i in e]) for e in gen]
        prod_pto_min = [sum([elemento_cono[i-1] for i in e]) for e in elem_min]
        minimo = min(prod_pto_min)
        repeticiones1 = countOf(prod_pto_gen+prod_pto_min, minimo)
        
        elemento_cono=sum([random.randint(1,100)*vector(rayo) for rayo in cono.rays()])
        prod_pto_gen = [sum([elemento_cono[i-1] for i in e]) for e in gen]
        prod_pto_min = [sum([elemento_cono[i-1] for i in e]) for e in elem_min]
        minimo = min(prod_pto_min)
        repeticiones2 = countOf(prod_pto_gen+prod_pto_min, minimo)
        
        repeticiones=min(repeticiones1,repeticiones2)
        if repeticiones > 1:
            gens_interesantes.append((gen, repeticiones))
    return gens_interesantes

#DESCOMENTAR PARA QUE EL OUTPUT SEA LA TABLA DE TODOS LOS DATOS
#         repeticiones = len(elem_min)+len(gen)
#         for rayo in cono.rays():
#             prod_pto_gen = [sum([rayo[i-1] for i in e]) for e in gen]
#             prod_pto_min = [sum([rayo[i-1] for i in e]) for e in elem_min]
#             minimo = min(prod_pto_min)
#             repeticiones = min(repeticiones, countOf(prod_pto_gen+prod_pto_min, minimo))
#         datos.append([gen, cono.dim(), repeticiones])
#     return table(datos, header_row=True)

Corre experimentos e imprime tabla con generadores de conjuntos no M-threshold e info del cono

In [25]:
ELEMENTOS = 9
gens=experimentos(ELEMENTOS,3, 3)
print(gens)
datos=[['Generadores', 'Elementos minimales en el complemento', 'Cant. igualdades', 'Secuencia de grado', 'Grado maximal']]
for gen in gens:
    P=generar_poset(ELEMENTOS,3)
    elem_min = encontrar_min(gen[0],P)
    cono = generar_cono(ELEMENTOS, gen[0], elem_min)
    cant_elementos, grado = calcular_grado(P, gen[0], ELEMENTOS)
    datos.append([gen[0], elem_min, gen[1], grado,  grado in dict_grados[cant_elementos]])
    
tabla = table(datos, header_row=True)
tabla

[({(2, 6, 7), (3, 4, 8), (1, 5, 9)}, 6), ({(4, 5, 7), (2, 3, 9), (1, 6, 8)}, 6), ({(3, 5, 7), (2, 4, 9), (1, 6, 8)}, 6), ({(2, 4, 9), (4, 5, 7), (1, 6, 8)}, 6), ({(4, 5, 6), (2, 3, 9), (1, 7, 8)}, 5), ({(4, 5, 7), (2, 3, 9), (1, 7, 8)}, 6), ({(2, 4, 9), (3, 5, 6), (1, 7, 8)}, 6), ({(2, 4, 9), (3, 5, 7), (1, 7, 8)}, 6), ({(2, 4, 9), (4, 5, 6), (1, 7, 8)}, 6), ({(2, 4, 9), (4, 5, 7), (1, 7, 8)}, 6)]


Tabla con igualdades en poset vs matroide

Generar cono dados elementos, # conjuntos y generadores específicos

In [87]:
#PARÁMETROS
ELEMENTOS = 9
CARD_CONJ = 3
GENERADORES = [(3,4,7),(4,5,6),(2,3,9),(1,7,8)]

P=generar_poset(ELEMENTOS,CARD_CONJ)
elem_min = encontrar_min(GENERADORES,P)
print(elem_min)
cono = generar_cono(ELEMENTOS, GENERADORES, elem_min)
cono

[(1, 4, 9), (2, 4, 8), (2, 5, 7)]


A 2-dimensional polyhedron in QQ^9 defined as the convex hull of 1 vertex, 1 ray, 1 line (use the .plot() method to plot)

Dados generadores específicos, evalúa en un elemento aleatorio del cono y cuenta igualdades

In [95]:
from operator import countOf
import random
#PARÁMETROS
ELEMENTOS = 9
CARD_CONJ = 3
GENERADORES = [(4,5,6),(2,4,7),(2,3,9),(1,7,8)]

#EXPERIMENTO
P=generar_poset(ELEMENTOS,CARD_CONJ)
elem_min = encontrar_min(GENERADORES,P)
cono = generar_cono(ELEMENTOS, GENERADORES, elem_min)
elemento=sum([random.randint(0,100)*vector(rayo) for rayo in cono.rays()])
# for i in range(ELEMENTOS):
#     elemento.append(sum([random.randint(1,100)*rayo[i] for rayo in cono.rays()]))
print(elemento)
print(cono.rays())
for gen in GENERADORES:
    a=sum([elemento[i-1] for i in gen])
    print(sum([elemento[i-1] for i in gen]))
print('----------')
for minimal in elem_min:
    print(sum([elemento[i-1] for i in minimal]))
print('***********')
elemento_por_conjs = [sum([elemento[i-1] for i in conj]) for conj in P.order_ideal(GENERADORES)]
print(countOf(elemento_por_conjs, a))
elemento_por_conjs_todo = [sum([elemento[i-1] for i in conj]) for conj in P.list()]
print(countOf(elemento_por_conjs_todo, a))
gen_ideal_cont=[]
for i in range(len(GENERADORES[0])):
    e= max(t[i] for t in GENERADORES)
    gen_ideal_cont.append(e)
ideal_cont = P.order_ideal([tuple(gen_ideal_cont)])
elemento_por_conjs_M = [sum([elemento[i-1] for i in conj]) for conj in ideal_cont]
print(countOf(elemento_por_conjs_M, a))    

len(P.order_ideal([(4,7,9)]))
# revisar en los otros, revisar si son degree max, uniquely realizable
# número de igualdades en todo el poset vs matroide
# 11 vértices y caras de tamaño 4 (conjetura simon)

(-102, -51, -51, 0, 0, 0, 51, 51, 102)
(A ray in the direction (-2, -1, -1, 0, 0, 0, 1, 1, 2),)
0
0
0
0
----------
0
0
0
0
***********
4
18
18


70